In [ ]:
#importing the needed libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import openpyxl

In [ ]:
# Load the Excel file
file_path = 'data.xlsx'
df = pd.read_excel(file_path)

In [ ]:
# Initialize the Selenium WebDriver (Ensure you have the correct WebDriver for your browser)
driver = webdriver.Chrome()  # You can use other drivers like FirefoxDriver or EdgeDriver

In [ ]:
# Function to get data from the website using a PIN number
def get_data_from_website(pin_number):
    driver.get("url")

    # Find the PIN input field and enter the PIN number
    pin_input = driver.find_element(By.NAME, "property_key")
    pin_input.clear()
    pin_input.send_keys(pin_number)
    pin_input.send_keys(Keys.RETURN)

    # Allow time for the page to load
    time.sleep(5)

    # Extract the required fields
    data = {}
    try:
        data['Tax Year'] = driver.find_element(By.XPATH, "//div[contains(text(), 'Tax Year')]/following-sibling::div/div").text
        data['Net Taxable Value'] = driver.find_element(By.XPATH, "//div[contains(text(), 'Net Taxable Value')]/following-sibling::div").text
        data['Tax status'] = driver.find_element(By.XPATH, "//div[contains(text(), 'Tax Status')]/following-sibling::div").text
        data['Comments'] = driver.find_element(By.XPATH, "//div[contains(text(), 'Owner Name & Address')]/following-sibling::div").text
        data['TAXCODE'] = driver.find_element(By.XPATH, "//div[contains(text(), 'Tax Code')]/following-sibling::div").text
        data['name of owner'] = driver.find_element(By.XPATH, "//div[contains(text(), 'Owner Name & Address')]/following-sibling::div").text
    except Exception as e:
        print(f"Error while extracting data for PIN {pin_number}: {e}")
    return data

In [ ]:
# Loop through each PIN in the Excel sheet and update the data
for index, row in df.iterrows():
    pin_number = row['PIN']
    if pd.isna(pin_number) or pin_number == '':
        print("Empty PIN encountered, stopping the loop.")
        break
    if pd.notna(row['Tax Year']):
        print(f"Data already present for PIN {pin_number}, skipping.")
        continue
    data = get_data_from_website(pin_number)
    print(data)
    for key, value in data.items():
        if key in df.columns:
            df.at[index, key] = value
            print(index)
        else:
            print("Not found")


In [ ]:
# Save the updated Excel file
df.to_excel(file_path, index=False)

# Close the WebDriver
driver.quit()